In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from scipy.interpolate import interp1d

import requests
from datetime import datetime
import os

d:\Learn\Guvi\DS\project\Air Gesture\air_env\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
# url = "192.168.0.109:8080"

SAMPLING_RATE = 100 
Z_DOWN_THRESHOLD = -7.0
Z_UP_THRESHOLD = 7.0
WINDOW_SIZE = 5
MIN_LEN = 30      # discard tiny segments
COOLDOWN = 10     # samples to ignore after a flip

In [3]:
def extract_series(buffer):
    """
    phyphox format:
    buffer = [[v1, v2, v3, ...]]
    We want:
    [v1, v2, v3, ...]
    """
    if isinstance(buffer, list) and len(buffer) == 1:
        return buffer[0]
    return buffer

def fetch_phyphox_data(url):
    PHY_PHOX_URL = (
    f"http://{url}/get?gyrX=full&gyrY=full&gyrZ=full&accX=full&accY=full&accZ=full"
)
    response = requests.get(PHY_PHOX_URL, timeout=10)
    response.raise_for_status()
    return response.json()["buffer"]

def parse_sensor_data(raw):
    # print(raw["buffer"])
     # Find common length
    acc_x = extract_series(raw["accX"]["buffer"])
    acc_y = extract_series(raw["accY"]["buffer"])
    acc_z = extract_series(raw["accZ"]["buffer"])

    gyro_x = extract_series(raw["gyrX"]["buffer"])
    gyro_y = extract_series(raw["gyrY"]["buffer"])
    gyro_z = extract_series(raw["gyrZ"]["buffer"])
    min_len = min(
        len(acc_x), len(acc_y), len(acc_z),
        len(gyro_x), len(gyro_y), len(gyro_z)
    )
    df = pd.DataFrame({
        "acc_x": acc_x[:min_len],
        "acc_y": acc_y[:min_len],
        "acc_z": acc_z[:min_len],
        "gyro_x": gyro_x[:min_len],
        "gyro_y": gyro_y[:min_len],
        "gyro_z": gyro_z[:min_len],
    })
    return df

def add_time(df):
    df["time"] = np.arange(len(df)) / SAMPLING_RATE
    return df

def trim_gesture(df):
    down = df["acc_z"] < Z_DOWN_THRESHOLD
    up = df["acc_z"] > Z_UP_THRESHOLD

    down_confirm = down.rolling(WINDOW_SIZE).sum() >= WINDOW_SIZE
    up_confirm = up.rolling(WINDOW_SIZE).sum() >= WINDOW_SIZE

    if not down_confirm.any():
        raise ValueError("❌ Flip DOWN not detected")

    start_idx = down_confirm.idxmax()

    up_after = up_confirm.loc[start_idx + 1:]
    if not up_after.any():
        raise ValueError("❌ Flip UP not detected")

    end_idx = up_after.idxmax()

    return df.loc[start_idx:end_idx].reset_index(drop=True)



In [4]:
def confirm_flip(signal, threshold, window, mode="down"):
    if mode == "down":
        cond = signal < threshold
    else:
        cond = signal > threshold

    return cond.rolling(window).sum() >= window


def split_on_flip(df):
    acc_z = df["acc_z"]

    down_confirm = confirm_flip(acc_z, Z_DOWN_THRESHOLD, WINDOW_SIZE, "down")
    up_confirm   = confirm_flip(acc_z, Z_UP_THRESHOLD,   WINDOW_SIZE, "up")

    segments = []
    i = 0
    n = len(df)

    while i < n:
        # find flip DOWN
        if down_confirm.iloc[i]:
            start = i

            # cooldown after DOWN
            j = i + COOLDOWN

            # find flip UP
            while j < n and not up_confirm.iloc[j]:
                j += 1

            if j >= n:
                break  # no closing UP → discard tail

            end = j

            segment = df.iloc[start:end].reset_index(drop=True)

            if len(segment) >= MIN_LEN:
                segments.append(segment)

            # move index past this gesture
            i = end + COOLDOWN
        else:
            i += 1

    if not segments:
        raise ValueError("❌ No valid gestures detected")

    return segments


In [6]:
model = load_model("../model/air_gesture_cnn.h5")
mean = np.load("../processed/norm_mean.npy")
std  = np.load("../processed/norm_std.npy")
SENSOR_COLS = ["acc_x", "acc_y", "acc_z", "gyro_x", "gyro_y", "gyro_z"]
TIMESTEPS = 150

In [7]:
def resample_gesture(df, timesteps=150):
    old_len = len(df)
    old_x = np.linspace(0, 1, old_len)
    new_x = np.linspace(0, 1, timesteps)

    data = []
    for col in SENSOR_COLS:
        f = interp1d(old_x, df[col].values, kind="linear")
        data.append(f(new_x))

    return np.stack(data, axis=1)  # (150, 6)

def normalize(X):
    return (X - mean) / std

def predict_digit(raw_df):
    """
    raw_df: DataFrame with acc & gyro columns for ONE gesture
    """
    df = raw_df[SENSOR_COLS]

    X = resample_gesture(df)
    X = normalize(X)
    X = np.expand_dims(X, axis=0)  # (1, 150, 6)

    probs = model.predict(X)
    digit = np.argmax(probs)

    return digit, probs

In [20]:
# Collect data from phyphox

print("📡 Fetching phyphox data...")
raw = fetch_phyphox_data("192.168.0.109:8080")

print("📊 Parsing sensor buffers...")
df = parse_sensor_data(raw)

print("⏱️ Creating timestamps...")
df = add_time(df)

print("✂️ Trimming gesture...")
# gesture_df = trim_gesture(df)
gesture_df = split_on_flip(df)

print(f"🤖 Detected {len(gesture_df)} gesture(s).")

📡 Fetching phyphox data...
📊 Parsing sensor buffers...
⏱️ Creating timestamps...
✂️ Trimming gesture...
🤖 Detected 3 gesture(s).


In [21]:
for i, gdf in enumerate(gesture_df):
    print(f"\n👉 Processing gesture {i+1}...")
    try:
        digit, probs = predict_digit(gdf)
        print(f"✅ Predicted digit: {digit}")
        print(f"   Probabilities: {probs.flatten()}")
    except Exception as e:
        print(f"❌ Prediction failed: {e}")
# predict_digit(raw_df)


👉 Processing gesture 1...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
✅ Predicted digit: 3
   Probabilities: [1.6975415e-18 9.2872471e-29 4.8388877e-19 1.0000000e+00 2.4415609e-26
 7.0154606e-15 7.7233504e-27 1.2204454e-21 2.4987317e-22 2.0444373e-22]

👉 Processing gesture 2...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
✅ Predicted digit: 4
   Probabilities: [3.7649764e-21 1.9695834e-23 3.6477468e-22 6.8423423e-15 1.0000000e+00
 7.1165654e-23 1.6118786e-18 5.6244828e-19 1.1928928e-13 8.4296228e-15]

👉 Processing gesture 3...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
✅ Predicted digit: 5
   Probabilities: [4.74349005e-17 2.28496970e-27 1.21840513e-33 3.38040327e-22
 3.84304930e-26 1.00000000e+00 2.81703272e-20 1.04414246e-26
 6.84586695e-30 1.96369721e-20]
